In [1]:
from numpydl.model import Network,DenseLayer
import numpy as np
from sklearn.preprocessing import LabelEncoder
import sklearn

model =Network()


from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)

def transform(x,y):
    y = LabelEncoder().fit_transform(y)
    return np.array(x), np.array(y)

x,y=transform(X,y)


model.add(DenseLayer(X.shape[0]))
model.add(DenseLayer(8))
model.add(DenseLayer(10))
model.add(DenseLayer(3))

model.train(X_train=x, y_train=y, epochs=200)

TypeError: __init__() missing 1 required positional argument: 'data'

In [6]:
X = np.random.uniform(low=-2, high=5, size=(6,4))
ytrue = np.array([1,0,1,2,0,2])


class DenseLayer:
    def __init__(self, neurons) -> None:
        self.neurons = neurons
        self.biases = np.zeros((1, neurons))
        self.activation = None

    def ReLU(self, inputs):
        return np.maximum(0, inputs)
    
    def Softmax(self, inputs):
        e_vals = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        return e_vals / np.sum(e_vals, axis=1, keepdims=True)
    
    def relu_derivative(self, dA, Z):
        dZ = np.array(dA, copy = True)
        dZ[Z <= 0] = 0;
        return dZ;

    def forward(self, inputs, last=False):
        self.weights = np.random.uniform(low=-1, high=1, size=(inputs.shape[1], self.neurons))
        if last == True:
            self.Z = np.dot(inputs, self.weights) + self.biases
            self.A = self.Softmax(self.Z)
            self.activation = 'softmax'
        else:
            self.Z = np.dot(inputs, self.weights) + self.biases
            self.A = self.ReLU(self.Z)
            self.activation = 'relu'
            
    def backward(self, dA_curr, W_curr, Z_curr, A_prev):
        m = A_prev.shape[1]

        dZ_curr = self.relu_derivative(dA_curr, Z_curr)
        dW_curr = np.dot(dZ_curr.T, A_prev) / m
        db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
        dA_prev = np.dot(W_curr, dZ_curr.T)

        return dA_prev, dW_curr, db_curr
    

class Network:
    def __init__(self, data):
        self.network = []
        self.memory = {}
        self.gradients = {}
        self.data = data
    
    def add(self, layer):
        self.network.append(layer)
    
    def _one_hot(self, labels):
        ohy = np.zeros((labels.size, labels.max() + 1))
        ohy[np.arange(labels.size), labels] = 1
        ohy_t = ohy.T
        return ohy_t
        
    def _calculate_loss(self, outputs, labels):
        samples = len(labels)

        out_clipped = np.clip(outputs, 1e-7, 1-1e-7)

        if len(labels.shape) == 1:
            confs = out_clipped[range(samples), labels]
        elif len(labels.shape) == 2:
            confs = np.sum(out_clipped*labels, axis=1)

        return np.mean(-np.log(confs))
    
    def _backprop(self, actual_y, predict_y):
        actual_y = self._one_hot(labels=actual_y)
        actual_y = actual_y.reshape(predict_y.shape)
        
        dA_prev = - (np.divide(actual_y, predict_y) - np.divide(1 - actual_y, 1 - predict_y))
        
        for layer_idx_prev, layer in reversed(list(enumerate(self.network))):
            layer_idx_curr = layer_idx_prev + 1
            
            if layer_idx_prev == 0:
                A_prev = self.data
            else:
                ## if output layer --> insert dC/y_hat, otherwise update with dC/dZ
                dA_curr = dA_prev

                A_prev = self.memory[layer_idx_prev]['A']
                Z_curr = self.memory[layer_idx_curr]['Z']
                W_curr = self.memory[layer_idx_curr]['W']
            
            dA_prev, dW_curr, db_curr = layer.backward(dA_curr, W_curr, Z_curr, A_prev)
            
            self.gradients[layer_idx_curr] = {'dW':dW_curr, 'db':db_curr}
    
    def _forwardprop(self):
        new_out = []
        for idx, layer in enumerate(self.network):
            if layer != self.network[-1]:
                if not new_out:
                    layer.forward(self.data)
                    new_out.append(layer.A)
                    self.memory[idx+1] = {'W':layer.weights, 'Z':layer.Z, 'A':layer.A,
                                         'b':layer.biases}
                else:
                    layer.forward(new_out[-1])
                    new_out.append(layer.A)
                    self.memory[idx+1] = {'W':layer.weights, 'Z':layer.Z, 'A':layer.A,
                                         'b':layer.biases}
            else:
                layer.forward(new_out[-1], last=True)
                new_out.append(layer.A)
                self.memory[idx+1] = {'W':layer.weights, 'Z':layer.Z, 'A':layer.A,
                                      'b':layer.biases}
        
        return new_out[-1]
    
    def _update(self, lr=0.01):
        for idx, layer in enumerate(self.network):
            idx = idx + 1
            self.memory[idx]['W'] -= learning_rate * self.gradients[idx]['dW']     
            self.memory[idx]['b'] -= learning_rate * self.gradients[idx]['db']
            
    def _get_accuracy(self, predicted, actual):
        return np.mean(np.argmax(predicted, axis=1)==actual)
            

model = Network(data=X)
model.add(DenseLayer(neurons=X.shape[0]))
model.add(DenseLayer(neurons=6))
model.add(DenseLayer(neurons=3))
out = model._forwardprop()

model._backprop(actual_y=ytrue, predict_y=out)

# loss = model.calculate_loss(outputs=out, labels=ytrue)

# print('PREDICTIONS:', np.argmax(out, axis=1))
# print('ACTUAL:', ytrue)
# print('LOSS:', loss)
# print('ACCURACY:', np.mean(np.argmax(out, axis=1)==ytrue))

In [8]:
loss = model._calculate_loss(outputs=out, labels=ytrue)
print('PREDICTIONS:', np.argmax(out, axis=1))
print('ACTUAL:', ytrue)
print('LOSS:', loss)
print('ACCURACY:', np.mean(np.argmax(out, axis=1)==ytrue))

PREDICTIONS: [0 0 0 0 0 0]
ACTUAL: [1 0 1 2 0 2]
LOSS: 2.953763258510348
ACCURACY: 0.3333333333333333
